In [85]:
dataset_urls = dict()

In [ ]:
# wn18rr datasets
dataset_urls['wn18rr-full'] = {
    'train': 'https://raw.githubusercontent.com/hvags/effective-octo-eureka/main/wn18rr/train_wn18rr.txt',
    'validate': 'https://raw.githubusercontent.com/hvags/effective-octo-eureka/main/wn18rr/valid_wn18rr.txt',
    'test': 'https://raw.githubusercontent.com/hvags/effective-octo-eureka/main/wn18rr/test_wn18rr.txt'
    }

In [86]:
dataset_urls['wn18rr-sym'] = {
    'train': 'https://raw.githubusercontent.com/hvags/effective-octo-eureka/main/wn18rr/sym_train_wn18rr.txt',
    'validate': 'https://raw.githubusercontent.com/hvags/effective-octo-eureka/main/wn18rr/sym_valid_wn18rr.txt',
    'test': 'https://raw.githubusercontent.com/hvags/effective-octo-eureka/main/wn18rr/sym_test_wn18rr.txt'
    }

In [ ]:
dataset_urls['wn18rr-asym'] = {
    'train': 'https://raw.githubusercontent.com/hvags/effective-octo-eureka/main/wn18rr/asym_train_wn18rr.txt',
    'validate': 'https://raw.githubusercontent.com/hvags/effective-octo-eureka/main/wn18rr/asym_valid_wn18rr.txt',
    'test':'https://raw.githubusercontent.com/hvags/effective-octo-eureka/main/wn18rr/asym_test_wn18rr.txt'
    }

In [77]:
# fb15k-237 datasets
dataset_urls['fb15k-237-full'] = {
    'train': 'https://raw.githubusercontent.com/hvags/effective-octo-eureka/main/fb15k-237/train.txt',
    'validate': 'https://raw.githubusercontent.com/hvags/effective-octo-eureka/main/fb15k-237/valid.txt',
    'test': 'https://raw.githubusercontent.com/hvags/effective-octo-eureka/main/fb15k-237/test.txt'
    }

dataset_urls['fb15k-237-sym'] = {
    'train': 'https://raw.githubusercontent.com/hvags/effective-octo-eureka/main/fb15k-237/sym_train.txt',
    'validate': 'https://raw.githubusercontent.com/hvags/effective-octo-eureka/main/fb15k-237/sym_valid.txt',
    'test': 'https://raw.githubusercontent.com/hvags/effective-octo-eureka/main/fb15k-237/sym_test.txt'
    }

dataset_urls['fb15k-237-asym'] = {
    'train': 'https://raw.githubusercontent.com/hvags/effective-octo-eureka/main/fb15k-237/asym_train.txt',
    'validate': 'https://raw.githubusercontent.com/hvags/effective-octo-eureka/main/fb15k-237/asym_valid.txt',
    'test': 'https://raw.githubusercontent.com/hvags/effective-octo-eureka/main/fb15k-237/asym_test.txt'
    }

In [87]:
models = [
#          'TransE',
          'TransH',
#          'TransD',
#          'TransR',
#          'RESCAL',
#          'ComplEx',
#          'RotatE'
         ]

In [88]:
import pandas as pd
from pykeen.pipeline import pipeline
from pykeen.triples import TriplesFactory

datasets = dict()

for key in dataset_urls.keys():
    print(f'Processing: {key}')
    
    datasets[key] = dict()
    
    df_train = pd.read_csv(dataset_urls[key]['train'], header=None, sep='\t', names=['head', 'relation','tail'])
    df_validate = pd.read_csv(dataset_urls[key]['validate'], header=None, sep='\t', names=['head', 'relation','tail'])
    df_test = pd.read_csv(dataset_urls[key]['test'], header=None, sep='\t', names=['head', 'relation','tail'])
    
    datasets[key]['train'] = TriplesFactory.from_labeled_triples(df_train.astype('str').to_numpy())
    entity_mapping = datasets[key]['train'].entity_to_id
    relation_mapping = datasets[key]['train'].relation_to_id
    
    datasets[key]['validate'] = TriplesFactory.from_labeled_triples(df_validate.astype('str').to_numpy(),
                                                                    entity_to_id=entity_mapping,
                                                                    relation_to_id=relation_mapping
                                                                    )
    
    datasets[key]['test'] = TriplesFactory.from_labeled_triples(df_test.astype('str').to_numpy(),
                                                                    entity_to_id=entity_mapping,
                                                                    relation_to_id=relation_mapping
                                                                    )
    print('\n')

Processing: wn18rr-sym


In [89]:
import requests
import json
from pykeen.pipeline import pipeline_from_config

HPO_URL_BASE = 'https://raw.githubusercontent.com/hvags/effective-octo-eureka/main/hpo_results'
HPO_URL_TAIL = 'best_pipeline/pipeline_config.json'

results = dict()

for dataset_name, dataset_url in dataset_urls.items():    
    for model in models:
        
        url = f'{HPO_URL_BASE}/{dataset_name}/{model}/{HPO_URL_TAIL}'        
        response = requests.get(url).text
        
        if (response.startswith('404')):
            print(dataset_name, model, url, response)
            continue
        
        config = json.loads(response)

        config['pipeline'].pop('testing')
        config['pipeline'].pop('training')
        config['pipeline'].pop('validation')        
        
        if (config['pipeline']['loss'] == 'marginranking'):
            if('margin_activation' in config['pipeline']['loss_kwargs']):
                config['pipeline']['loss_kwargs'].pop('margin_activation')
        
        dataset = datasets[dataset_name]
        
        results[dataset_name, model] = pipeline_from_config(config,
                              device='gpu',
                              testing=dataset['test'],
                              training=dataset['train'],
                              validation=dataset['validate'])

Training epochs on cuda:   0%|          | 0/100 [00:00<?, ?epoch/s]

INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=64.


Evaluating on cuda:   0%|          | 0.00/1.14k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 2.58s seconds


In [35]:
results[('wn18rr-full', 'TransH')]

PipelineResult(random_seed=4183616286, model=TransH(
  (loss): MarginRankingLoss(
    (margin_activation): ReLU()
  )
  (regularizer): TransHRegularizer()
  (entity_embeddings): Embedding(
    (_embeddings): Embedding(40559, 96)
  )
  (relation_embeddings): Embedding(
    (_embeddings): Embedding(11, 96)
  )
  (normal_vector_embeddings): Embedding(
    (_embeddings): Embedding(11, 96)
  )
), training=TriplesFactory(num_entities=40559, num_relations=11, num_triples=86835, inverse_triples=False), training_loop=<pykeen.training.slcwa.SLCWATrainingLoop object at 0x00000182800C8AF0>, losses=[54.79743130184753, 0.00015346237986022042, 0.00012516252601703378, 7.441305179645918e-05, 6.736971426347847e-05, 6.872640763181901e-05, 6.515155007694095e-05, 6.698185502161474e-05, 9.18062512015352e-05, 9.053287331686054e-05, 0.0001052353556648179, 0.00010482694441230074, 0.00010036756252627468, 0.00012549248017190947, 0.00016363166543344228, 0.0002498997786680418, 0.0002248443989595299, 0.000303000133

In [90]:
def results_to_table(results):
  
    metrics = [('both.realistic.mean_reciprocal_rank', 'MRR'),                  
                ('both.realistic.hits@1', 'Hits@1'),                
                ('both.realistic.hits@10', 'Hits@10')]         
   
    
    for dataset in dataset_urls.keys():
        print(dataset)
        table = pd.DataFrame()
       
        for model_label in models:                            
            row = [model_label]
            
            for metric in metrics:      
                if ((dataset, model_label) not in results):
                    result = 'NaN'
                    row += [result]
                else:
                    result = results[(dataset, model_label)].get_metric(metric[0])
                    row += [round(result,5)]                
            table = table.append(pd.DataFrame([row], columns =['model'] + [metric[1] for metric in metrics]))

        print(table.to_string(index=False))
        print('\n')


results_to_table(results)

wn18rr-sym
 model     MRR  Hits@1  Hits@10
TransH 0.05384     0.0  0.13716


